<a href="https://colab.research.google.com/github/jurandi82/GrafoSocietario/blob/main/GrafoSocietario2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Consulta Grafo Societário

In [1]:
import pandas as pd
import networkx as nx
import ipywidgets as wd
import sqlalchemy as sa
from tqdm.notebook import tqdm

#uri_sqlite ='sqlite:///base_receita20230315.db'
uri_sqlite ='sqlite:///grapho20230315.db'
engine = sa.create_engine(uri_sqlite)
engine.connect().close()

In [8]:
G = nx.Graph()
print("Lendo database... leva cerca de 2 minutos")
with engine.connect() as db:
    query ='''SELECT COUNT() FROM grapho;'''
    total = db.execute(sa.text(query)).fetchall()[0][0]
    query ='''SELECT A,B FROM grapho limit 10;'''
    cursor = db.execute(sa.text(query))
    for i in tqdm(cursor,total=total):
        G.add_edge(*i)
print("Número de registros carregados: {:,}".format(G.number_of_nodes()).replace(",","."))

Lendo database... leva cerca de 2 minutos


  0%|          | 0/22708202 [00:00<?, ?it/s]

Número de registros carregados: 16


In [39]:
def validar(pesquisa):
    for node in tqdm(G.nodes()):
        if node.startswith(pesquisa):
            return node
    return ""

In [47]:
def localizar(inicio, fim):
    inicio, fim = validar(inicio), validar(fim)
    print(f" Inicio validado: {inicio} \n Fim validado {fim}\n")
    if inicio !="" and fim !="":
        if not nx.has_path(G,inicio ,fim):
            print("Caminho não encontrado")
        else:
            print("Caminho localizado \n"+"*"*64)
            print(*nx.shortest_path(G,source=fim,target=inicio),sep = "\n")
        print("\n")

In [50]:
lb_alvo = wd.Label("ALVO:")
tx_alvo = wd.Text(value='TELEFONICA')
box_alvo=wd.HBox([lb_alvo,tx_alvo])

lb_pesquisa = wd.Label("ORIGEM:")
tx_pesquisa = wd.Text(value='NESTLE')
bt_pesquisa = wd.Button(description="PESQUISAR", icon='search')
out = wd.Output(layout={'border': '1px solid black'})
box_origem=wd.HBox([lb_pesquisa,tx_pesquisa,bt_pesquisa])
box=wd.VBox([box_alvo,box_origem,out])

def executa(self):
    with out:
        out.clear_output()
        localizar(tx_alvo.value,tx_pesquisa.value)

bt_pesquisa.on_click(executa)

# testes

In [51]:
print("Sugestoes de pesquisa: Nestle, Telefonica,Odebrecht,Wassef...\n")
display(box)

Sugestoes de pesquisa: Nestle, Telefonica,Odebrecht,Wassef...



# Antigo

## Esta aplicação demonstra a funcionalidade da estrutura de grafos na pesquisa de relações
### Para este código foi utilizado o recorte de 2 milhões de Empresas e Socios
#### O processamento da base total permite encontrar relações societárias entre quaisquer empresas. <br> Podemos definir a profundidade de pesquisa de modo a encontrar empresas com indicio de "ponte"

In [4]:
import pandas as pd
import networkx as nx
import ipywidgets as wd

arquivo='socio_graph_2020ABR.gz'
#arquivo='/content/drive/MyDrive/socio_graph_2020ABR.gz'

print("Lendo arquivo... leva cerca de 40 segundos")
G = nx.read_edgelist(arquivo, delimiter='\t', encoding='utf-8')
print("Número de registros carregados: {:,}".format(G.number_of_nodes()).replace(",","."))
print("Gerando matriz de dados... leva outros 40 segundos")
dataset=nx.to_pandas_edgelist(G)

def validar(pesquisa):
    from sys import exit
    h=dataset[dataset.source.str.contains(pesquisa.upper())]
    if h.size==0:
        h=dataset[dataset.target.str.contains(pesquisa.upper())]
    if h.size==0:
        exit("Não localizado")
    return h.iloc[0][0]

def localizar(inicio, fim):
    inicio, fim = validar(inicio), validar(fim)
    if not nx.has_path(G,inicio ,fim):
        print("Caminho não encontrado")
    else:
        print("Caminho localizado \n"+"*"*64)
        print(*nx.shortest_path(G,source=fim,target=inicio),sep = "\n")
    print("\n")

print("Pronto para seguir\n")

lb_alvo = wd.Label("ALVO:")
tx_alvo = wd.Text(value='TELEFONICA')
box_alvo=wd.HBox([lb_alvo,tx_alvo])

lb_pesquisa = wd.Label("ORIGEM:")
tx_pesquisa = wd.Text(value='NESTLE')
bt_pesquisa = wd.Button(description="PESQUISAR", icon='search')
out = wd.Output(layout={'border': '1px solid black'})
box_origem=wd.HBox([lb_pesquisa,tx_pesquisa,bt_pesquisa])
box=wd.VBox([box_alvo,box_origem,out])

def executa(self):
    with out:
        out.clear_output()
        localizar(tx_alvo.value,tx_pesquisa.value)

bt_pesquisa.on_click(executa)

Lendo arquivo... leva cerca de 40 segundos
Número de registros carregados: 2.022.573
Gerando matriz de dados... leva outros 40 segundos
Pronto para seguir



## Use a caixa abaixo para pesquisa

In [5]:
print("Sugestoes de pesquisa: Nestle, Telefonica,Odebrecht,Wassef...\n")
display(box)

Sugestoes de pesquisa: Nestle, Telefonica,Odebrecht,Wassef...

